<h1>Loading the dataset and cleaning </h1>

In [3]:
import numpy as np 
import pandas as pd
from sklearn.naive_bayes import GaussianNB
path='/Users/vjsamuel/kode-monkeys/assignment_4_Logistic_regression/autos.csv'
data = pd.read_csv(path, sep=',', encoding='Latin1')
data[:10]

,datecrawled,name,seller,offertype,price,abtest,vehicletype,yearofregistration,gearbox,powerps,model,kilometer,monthofregistration,fueltype,brand,notrepaireddamage,datecreated,nrofpictures,postalcode,lastseen
0,2016-03-24T11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24T00:00:00,False,70435,2016-04-07T03:16:57
1,2016-03-24T10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24T00:00:00,False,66954,2016-04-07T01:46:50
2,2016-03-14T12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14T00:00:00,False,90480,2016-04-05T12:47:46
3,2016-03-17T16:54:04,GOLF_4_1_4__3TÃRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17T00:00:00,False,91074,2016-03-17T17:40:17
4,2016-03-31T17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31T00:00:00,False,60437,2016-04-06T10:17:21
5,2016-04-04T17:36:23,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,privat,Angebot,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04T00:00:00,False,33775,2016-04-06T19:17:07
6,2016-04-01T20:48:51,Peugeot_206_CC_110_Platinum,privat,Angebot,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein,2016-04-01T00:00:00,False,67112,2016-04-05T18:18:39
7,2016-03-21T18:54:38,VW_Derby_Bj_80__Scheunenfund,privat,Angebot,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein,2016-03-21T00:00:00,False,19348,2016-03-25T16:47:58
8,2016-04-04T23:42:13,Ford_C___Max_Titanium_1_0_L_EcoBoost,privat,Angebot,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN,2016-04-04T00:00:00,False,94505,2016-04-04T23:42:13
9,2016-03-17T10:53:50,VW_Golf_4_5_tuerig_zu_verkaufen_mit_Anhaengerk...,privat,Angebot,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17T00:00:00,False,27472,2016-03-31T17:17:06


In [4]:
# Remove nrofpictures column
work_data=data.drop('nrofpictures',1)
print(data.groupby('seller').size())

seller
gewerblich         3
privat        371525
dtype: int64


In [5]:
# Remove seller column
work_data = work_data[work_data.seller != 'gewerblich']
work_data=work_data.drop('seller',1)
print(work_data.groupby('offertype').size())

offertype
Angebot    371513
Gesuch         12
dtype: int64


In [6]:
# Remove offertype column
work_data = work_data[work_data.offertype != 'Gesuch']
work_data=work_data.drop('offertype',1)
#inspecting: name
print(len(work_data.groupby('name').size()), 'different names for all offers')

233517 different names for all offers


In [7]:
# Remove name and abtest column
work_data=work_data.drop('name',1)
work_data=work_data.drop('abtest',1)

In [8]:
# Drop NA inplace
work_data.dropna(inplace=True)

In [16]:
# Create categorical for vehicle type
work_data.vehicletype = pd.Categorical(work_data.vehicletype)
work_data['vehicletype_coded'] = work_data.vehicletype.cat.codes
vehicle_array = work_data['vehicletype_coded'].tolist()
vehicle_array[:5]

[4, 4, 6, 2, 6]

In [17]:
# Create categorical for brand type
work_data.brand = pd.Categorical(work_data.brand)
work_data['brand_coded'] = work_data.brand.cat.codes
brand_array = work_data['brand_coded'].tolist()
brand_array[:5]

[37, 31, 2, 25, 37]

In [18]:
# Create categorical for model type
work_data.model = pd.Categorical(work_data.model)
work_data['model_coded'] = work_data.model.cat.codes
model_array = work_data['model_coded'].tolist()
model_array[:5]

[118, 103, 11, 8, 40]

In [19]:
input = zip(brand_array, model_array)
ilist = list(input)
ilist[:5]

[(37, 118), (31, 103), (2, 11), (25, 8), (37, 40)]

In [24]:
gnb = GaussianNB()
gnb.fit(ilist, vehicle_array)
prediction = gnb.predict([[37, 118]])
print(work_data.vehicletype[prediction])

4    kleinwagen
Name: vehicletype, dtype: category
Categories (8, object): [andere, bus, cabrio, coupe, kleinwagen, kombi, limousine, suv]


In [22]:
gnb.score(ilist, vehicle_array)

0.35852078942326115